In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import numba
numba.config.DISABLE_JIT = True # j'ai une erreur quand je ne vide pas le cahche de numba, probablement corrompu
import dcor
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
import numpy as np  # Nouvelle bibliothèque pour générer des modèles quadratiques et autres
from sklearn.metrics import mutual_info_score
import itertools
from statsmodels.stats.outliers_influence import reset_ramsey
from statsmodels.stats.diagnostic import linear_reset
from statsmodels.stats.stattools import durbin_watson


In [2]:


# 1. Fonction pour le test de normalité
def test_normality(df, column):
    stat, p = stats.shapiro(df[column].dropna())
    is_normal = p > 0.05
    print(f"Test de normalité pour '{column}' : {'Normale' if is_normal else 'Non Normale'}")
    return is_normal

# 2. Test de linéarité : Durbin-Watson
def test_linearity(df, var1, var2):
    X = sm.add_constant(df[var1])
    model = sm.OLS(df[var2], X).fit()
    dw_stat = sm.stats.durbin_watson(model.resid)
    is_linear = (1.5 <= dw_stat <= 2.5)  # Indique l'absence d'autocorrélation (proche de 2)
    print(f"Test Durbin-Watson pour '{var1}' et '{var2}' : {'Linéaire' if is_linear else 'Non Linéaire'}")
    return is_linear

# 3. Test de non-linéarité : RESET de Ramsey
def test_non_linearity(df, var1, var2):
    X = sm.add_constant(df[var1])
    model = sm.OLS(df[var2], X).fit()
    ramsey_test = reset_ramsey(model, degree=2)
    is_nonlinear = ramsey_test.pvalue < 0.05
    print(f"Test RESET de Ramsey pour '{var1}' et '{var2}' : {'Non Linéaire' if is_nonlinear else 'Pas de non-linéarité détectée'}")
    return is_nonlinear

# 4. Test pour vérifier la présence de "ties" dans Kendall
def detect_ties(df, var1, var2):
    kendall_corr, p_value = stats.kendalltau(df[var1], df[var2])
    ties_count = len(df[var1].duplicated()) + len(df[var2].duplicated())
    print(f"Nombre de ties détectés : {ties_count}")
    return ties_count > 0

# 5. Fonction pour évaluer la distribution
def desequilibre_outliers(var_des):
    """
    Détecte les déséquilibres dans les fréquences des modalités et les valeurs aberrantes.

    Args:
        data : Série de données à analyser.

    Returns:
        dict : Résultats de l'analyse des déséquilibres.
    """
    # Vérification de la présence de données non nulles
    if var_des.isnull().all():
        return {"is_balanced": False, "skewness": np.nan, "num_outliers": 0}

    # Calcul des fréquences des modalités
    value_counts = var_des.value_counts()
    total_counts = len(var_des)
    
    # Vérification de l'inégalité des fréquences
    max_frequency = value_counts.max()
    min_frequency = value_counts.min()

    # Calcul de l'asymétrie
    skewness = stats.skew(var_des.dropna())

    # Détection des valeurs aberrantes
    Q1 = np.percentile(var_des.dropna(), 25)  # Premier quartile
    Q3 = np.percentile(var_des.dropna(), 75)  # Troisième quartile
    IQR = Q3 - Q1  # Intervalle interquartile
    
    # Limites pour les valeurs aberrantes
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identification des valeurs aberrantes
    outliers = var_des[(var_des < lower_bound) | (var_des > upper_bound)]

    # Décision basée sur les résultats
    decisions = {
        "is_balanced": (max_frequency / min_frequency) < 10,
        "skewness": skewness,
        "num_outliers": len(outliers)  # Nombre de valeurs aberrantes
    }
    
    return decisions


# 5. Fonction pour choisir la méthode de corrélation automatiquement
def choisir_correlation_auto(df, var1, var2, normal_var1, normal_var2):
    """
    Choisit la méthode de corrélation appropriée entre deux variables.

    Args:
        df : DataFrame contenant les données.
        var1 : Nom de la première variable.
        var2 : Nom de la deuxième variable.

    Returns:
        str : Méthode de corrélation sélectionnée (pearson, spearman ou kendall).
    """
    # Vérification de la présence des colonnes
    if var1 not in df.columns or var2 not in df.columns:
        raise ValueError(f"Les variables '{var1}' ou '{var2}' ne sont pas présentes dans le DataFrame.")

    # Cas où les deux variables sont normales
    if normal_var1 and normal_var2:
        if test_linearity(df, var1, var2):
            return "pearson"
        else:
            return "spearman"

    # Cas où l'une ou les deux variables ne sont pas normales
    if test_non_linearity(df, var1, var2):
        if desequilibre_outliers(df[var1])['is_balanced'] or desequilibre_outliers(df[var2])['is_balanced']:
            return "kendall"

    return "spearman"  # Méthode par défaut

# 6. Fonction pour calculer la corrélation en fonction de la méthode choisie
def calculer_correlation(method, df, var1, var2):
    if method == "pearson":
        return df[var1].corr(df[var2], method='pearson')
    elif method == "spearman":
        return df[var1].corr(df[var2], method='spearman')
    elif method == "kendall":
        return df[var1].corr(df[var2], method='kendall')
    elif method == "Quadratique":
        df['var1_squared'] = df[var1] ** 2
        X = sm.add_constant(df[[var1, 'var1_squared']])
        model = sm.OLS(df[var2], X).fit()
        return model.rsquared  # Retourne R² pour l'ajustement quadratique
    elif method == "Distance Corrélation":
        return dcor.distance_correlation(df[var1].values, df[var2].values)
    else:
        raise ValueError(f"Méthode de corrélation '{method}' non reconnue.")

# 7. Fonction pour la visualisation de la relation entre deux variables
from statsmodels.stats.diagnostic import linear_reset
from statsmodels.stats.stattools import durbin_watson

def visualiser_relation(df, var1, var2, mode='manuel'):
    """
    Cette fonction visualise et détecte automatiquement la relation entre deux variables (linéaire, quadratique, cubique).
    En mode manuel, l'utilisateur peut choisir le type de relation.
    En mode automatique, la fonction détecte le meilleur ajustement via des tests statistiques.
    """
    
    # Mode automatique : pas de visualisation, juste une détection automatique
    if mode == 'auto':
        X = sm.add_constant(df[var1])
        y = df[var2]
        
        # Test de Durbin-Watson pour détecter une relation linéaire
        model = sm.OLS(y, X).fit()
        dw_stat = durbin_watson(model.resid)
        
        # Test RESET pour détecter des non-linéarités
        reset_test = linear_reset(model, power=2, use_f=False)
        
        print(f"Durbin-Watson: {dw_stat}, Test RESET p-value: {reset_test.pvalue}")
        
        # Si DW proche de 2 et RESET non significatif, on peut conclure à une relation linéaire
        if 1.8 <= dw_stat <= 2.2 and reset_test.pvalue > 0.05:
            return '1'  # Relation linéaire
        elif reset_test.pvalue < 0.05:
            # Test quadratique
            poly_fit = np.poly1d(np.polyfit(df[var1], df[var2], 2))
            r2_quad = np.corrcoef(df[var2], poly_fit(df[var1]))[0, 1] ** 2
            poly_fit_cubic = np.poly1d(np.polyfit(df[var1], df[var2], 3))
            r2_cubic = np.corrcoef(df[var2], poly_fit_cubic(df[var1]))[0, 1] ** 2
            
            # Comparaison R² quadratique et cubique
            if r2_quad > r2_cubic:
                return '2'  # Relation quadratique
            else:
                return '3'  # Relation cubique
        else:
            return '4'  # Non-linéaire ou autre

    # Mode manuel : visualisation et choix manuel par l'utilisateur
    else:
        # Création de la figure avec plusieurs sous-graphiques
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))

        # 1. Ajustement linéaire
        sns.regplot(x=df[var1], y=df[var2], ax=axes[0], line_kws={"color": "red"}, ci=None)
        axes[0].set_title("Ajustement Linéaire")
        axes[0].set_xlabel(var1)
        axes[0].set_ylabel(var2)

        # 2. Ajustement quadratique
        poly_fit = np.poly1d(np.polyfit(df[var1], df[var2], 2))
        sns.scatterplot(x=df[var1], y=df[var2], ax=axes[1])
        sns.lineplot(x=df[var1], y=poly_fit(df[var1]), ax=axes[1], color="green")
        axes[1].set_title("Ajustement Quadratique")
        axes[1].set_xlabel(var1)
        axes[1].set_ylabel(var2)

        # 3. Ajustement cubique
        poly_fit_cubic = np.poly1d(np.polyfit(df[var1], df[var2], 3))
        sns.scatterplot(x=df[var1], y=df[var2], ax=axes[2])
        sns.lineplot(x=df[var1], y=poly_fit_cubic(df[var1]), ax=axes[2], color="blue")
        axes[2].set_title("Ajustement Cubique")
        axes[2].set_xlabel(var1)
        axes[2].set_ylabel(var2)

        plt.suptitle(f"Comparaison des Ajustements pour {var1} et {var2}")
        plt.tight_layout()
        plt.show()

        # Demande à l'utilisateur de choisir le type de relation
        print("\nQuel type de relation vous semble le plus adapté entre '{}' et '{}' ?".format(var1, var2))
        print("1 : Ajustement Linéaire")
        print("2 : Ajustement Quadratique")
        print("3 : Ajustement Cubique")
        print("4 : Non Linéaire ou Autre")
        relation_type = input("Choisissez une option (1, 2, 3 ou 4) : ")

        return relation_type
# 8. Fonction pour remplacer la probabilité par des étoiles
def obtenir_significativite(p_value):
    """
    Retourne les étoiles en fonction de la p-value.
    
    Args:
    p_value : La p-value obtenue du test statistique
    
    Retourne :
    Un string représentant le niveau de significativité ('***', '**', '*', ou '').
    """
    if p_value < 0.001:
        return '***'
    elif p_value < 0.01:
        return '**'
    elif p_value < 0.05:
        return '*'
    else:
        return ''
    
# 9. Fonction principale d'analyse de la corrélation (en mode automatique ou manuel)
def analyser_correlation(df, var1, var2, mode='auto'):
    """
    Analyse la corrélation entre deux variables.
    En mode 'auto', la fonction détecte automatiquement la méthode de corrélation.
    En mode 'manuel', l'utilisateur peut choisir la méthode après visualisation.
    
    Args:
    df : DataFrame contenant les données
    var1 : première variable (nom de la colonne)
    var2 : deuxième variable (nom de la colonne)
    mode : 'auto' pour un choix automatique de la méthode, 'manuel' pour choisir manuellement après visualisation.
    """
    
    # Test de normalité pour les deux variables
    normal_var1 = test_normality(df, var1)
    normal_var2 = test_normality(df, var2)

    # Choisir la méthode de corrélation (auto ou manuel)
    if mode == 'auto':
        # Choix automatique de la méthode de corrélation
        method = choisir_correlation_auto(df, var1, var2, normal_var1, normal_var2)
    else:
        # Visualisation et choix manuel
        visualiser_relation(df, var1, var2)
        print("Choisissez la méthode de corrélation :")
        print("1 : Pearson (linéaire, variables normales)")
        print("2 : Spearman (monotone, pas nécessairement normal)")
        print("3 : Kendall (si ties présents ou relation complexe)")
        print("4 : Distance Corrélation (relation non-linéaire)")
        choix = input("Entrez le numéro de la méthode : ")
        
        method = {
            '1': 'pearson',
            '2': 'spearman',
            '3': 'kendall',
            '4': 'Distance Corrélation'
        }.get(choix, 'pearson')  # Par défaut, Pearson
    
    # Calcul de la corrélation avec la méthode choisie
    correlation_value = calculer_correlation(method, df, var1, var2)
    print(f"Corrélation ({method}) entre {var1} et {var2} : {correlation_value}")
    
    return correlation_value, method


# 10. Fonction pour analyser plusieurs paires de variables


def analyser_correlation_multiple(df, variables, mode='auto'):
    """
    Analyse les corrélations entre toutes les combinaisons de variables dans une liste
    et stocke les résultats dans un DataFrame.
    
    Args:
    df : DataFrame contenant les données
    variables : Liste des noms des colonnes à analyser
    mode : 'auto' pour choisir automatiquement la méthode, 'manuel' pour choisir manuellement
    
    Retourne : un DataFrame contenant les résultats de la corrélation avec les colonnes demandées.
    """

    resultats = []

    # Générer toutes les combinaisons possibles de variables
    combinaisons = itertools.combinations(variables, 2)

    for var1, var2 in combinaisons:
        print(f"\nAnalyse de la corrélation entre '{var1}' et '{var2}' :")

        # Test de normalité pour les deux variables
        normal_var1 = test_normality(df, var1)
        normal_var2 = test_normality(df, var2)

        # Choix automatique ou manuel de la méthode de corrélation
        if mode == 'auto':
            method = choisir_correlation_auto(df, var1, var2, normal_var1, normal_var2)
        else:
            visualiser_relation(df, var1, var2)
            print("Choisissez la méthode de corrélation :")
            print("1 : Pearson (linéaire, variables normales)")
            print("2 : Spearman (monotone, pas nécessairement normal)")
            print("3 : Kendall (si ties présents ou relation complexe)")
            print("4 : Distance Corrélation (relation non-linéaire)")
            choix = input("Entrez le numéro de la méthode : ")
            method = {
                '1': 'pearson',
                '2': 'spearman',
                '3': 'kendall',
                '4': 'Distance Corrélation'
            }.get(choix, 'pearson')  # Par défaut Pearson

        # Calcul de la corrélation en fonction de la méthode
        if method == "pearson":
            corr, p_value = stats.pearsonr(df[var1], df[var2])
        elif method == "spearman":
            corr, p_value = stats.spearmanr(df[var1], df[var2])
        elif method == "kendall":
            corr, p_value = stats.kendalltau(df[var1], df[var2])
        elif method == "Distance Corrélation":
            corr = dcor.distance_correlation(df[var1].values, df[var2].values)
            p_value = np.nan  # La distance corrélation ne donne pas de p-value directement
        else:
            corr, p_value = np.nan, np.nan  # Si la méthode n'est pas reconnue

        # Obtenir la significativité sous forme d'étoiles
        significativite = obtenir_significativite(p_value)

        # Stocker les résultats dans une liste
        resultats.append({
            'Variable 1': var1,
            'Variable 2': var2,
            'Valeur de Corrélation': corr,
            'P-value': p_value,
            'Méthode de Corrélation': method,
            'Significativité': significativite
        })
    
    # Créer un DataFrame à partir des résultats
    df_resultats = pd.DataFrame(resultats, columns=['Variable 1', 'Variable 2', 'Valeur de Corrélation', 'P-value', 'Méthode de Corrélation', 'Significativité'])
    
    return df_resultats


In [ ]:
# Exemple d'utilisation
df = pd.read_csv('base_excel/data_scaled_zscore.csv')

analyser_correlation(df, 'key_sin', 'key_cos', mode="auto")

In [ ]:
# Exemple d'utilisation
df = pd.read_csv('base_excel/data_scaled_zscore.csv')
liste_variable = ['danceability', 'energy', 'mode', 'acousticness', 'valence',
       'tempo', 'release_year', 'key_sin', 'key_cos', 'loudness_winsorized',
       'speechiness_log', 'instrumentalness_log', 'liveness_log',
       'duration_log']
result_correlation = analyser_correlation_multiple(df, liste_variable, mode="auto")

'valence',
       'tempo', 'release_year', 'key_sin', 'key_cos', 'loudness_winsorized',
       'speechiness_log', 'instrumentalness_log', 'liveness_log',
       'duration_log'